In [1]:
import creative_api_pipeline_funcs as fn

import pandas as pd
import numpy as np

import os
import glob
from PIL import Image

import utils.ssl_setup as ssl

In [ ]:
data_period = '2025-01-01_2025-01-08'

cookie = ''
data_backup_path = f'../data/{data_period}'
bo_report_path = '../All Campaign Summary.csv'
asset_id_path = '../asset_id.parquet'
click_report_path = '../click_label_report.csv'

pod_height_path = '../pod_height.parquet'
unmatched_path = '../click_report_unmatched.parquet'

html_path = '../staging/html'
original_img_path = '../staging/img_original'
transformed_img_path = '../staging/img_transformed'
text_folder_path = '../staging/img_extracted_text/'

link_to_tesseract = r'C:\Users\user\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
service_account_json = '../../../../docs/api_key/xxx.json'

In [ ]:
fn.empty_folder(original_img_path, transformed_img_path, text_folder_path, text_folder_path)

In [6]:
df = fn.clean_bo_report(bo_report_path)

In [8]:
fn.get_df_assetid(df, asset_id_path, cookie)

In [ ]:
fn.store_df_image(asset_id_path, html_path, original_img_path, transformed_img_path, pod_height_path, cookie)

Fail to fetch HTML for 0000203646_SG
https://seadigitasacct.blob.core.windows.net/public/SEPCO/SEPCO%20REGION/PARADIGM/W1/PHF1H25-PreCRM--01ENNRN--ROC----------FS/Desktop/01_paradigm_security_cta_w1-churn.jpg download failed!
https://seadigitasacct.blob.core.windows.net/public/SEPCO/SEPCO%20REGION/PARADIGM/W1/PHF1H25-PreCRM--01ENNRN--ROC----------FS/Desktop/04_paradigm_galaxyai_cta_w1-churn.jpg download failed!
https://seadigitasacct.blob.core.windows.net/public/SEPCO/SEPCO%20REGION/PARADIGM/W1/PHF1H25-PreCRM--01ENNRN--ROC----------FS/Desktop/05_paradigm_galaxyai_cta_w1-churn.jpg download failed!
https://seadigitasacct.blob.core.windows.net/public/SEPCO/SEPCO%20REGION/PARADIGM/W1/PHF1H25-PreCRM--01ENNRN--ROC----------FS/Desktop/06_paradigm_galaxyai_cta_w1-churn.jpg download failed!
https://seadigitasacct.blob.core.windows.net/public/SEPCO/SEPCO%20REGION/PARADIGM/W1/PHF1H25-Pre-CRM-01ENNRN--AIOHD--------FS/Desktop/01_paradigm_security_cta_w1-AIdevice.jpg download failed!
https://seadigi

In [9]:
df_click_report = fn.process_click_report(click_report_path, pod_height_path, unmatched_path)

In [10]:
fn.update_bq_click_report_table(df_click_report, service_account_json)

In [3]:
fn.extract_text(img_path = original_img_path, text_folder_path = text_folder_path, link_to_tesseract = link_to_tesseract)

In [4]:
transformed_img_path = '../staging/img_transformed'
fn.qc_check(html_path, original_img_folder = original_img_path, transformed_img_folder = transformed_img_path, text_folder_path = text_folder_path)

original image files: 78
original image files: 78
Transformed image files: 78
Text files: 78
Perfect, no error.


In [ ]:
# height_dict = fn.get_img_height(transformed_img_folder = transformed_img_path)

In [5]:
fn.update_bq_campaign_asset_table(asset_id_path, service_account_json)

In [6]:
#upload original img
file_upload_list = glob.glob(f'{original_img_path}/*')

for file in file_upload_list:
    cid = file[-17:-7]
    country = file[-6:-4].lower()
    blob_name = f'{country}/{cid}.jpg'
    
    fn.upload_to_bucket(bucket_name='creative-edm-original', blob_name=blob_name, path_to_file=file, service_account_json=service_account_json)

In [7]:
#upload transformed img and metadata
file_upload_list = glob.glob(f'{transformed_img_path}/*')

for file in file_upload_list:
    cid = file[-17:-7]
    country = file[-6:-4].lower()
    blob_name = f'{country}/{cid}.jpg'

    fn.upload_to_bucket(bucket_name = 'creative-edm', blob_name = blob_name, path_to_file = file, service_account_json=service_account_json)


In [8]:
#upload extracted text
file_upload_list = glob.glob(f'{text_folder_path}/*')

for file in file_upload_list:
    cid = file[-17:-7]
    country = file[-6:-4].lower()
    blob_name = f'{country}/{cid}.txt'

    fn.upload_to_bucket(bucket_name='creative-edm-text', blob_name=blob_name, path_to_file=file, service_account_json=service_account_json)

In [5]:
fn.update_bq_campaigns_table(bo_report_path, service_account_json)